In [1]:
########### Python 2.7 #############
import urllib
import requests, json
import lxml.html
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient 
import pandas as pd
import json
import pprint
from tqdm import tqdm
from IPython.display import clear_output


class IndeedCrawl(object):

    def __init__(self,data_table):
        #self.query = "jobtitle:({0})".format(company_name)
        
        self.endpoint = 'http://api.indeed.com/ads/apisearch?'
        self.key = "6437595457989494"
       # self.client = MongoClient("mongodb://127.0.0.1:27017")
      #  self.db=self.client.jobs[data_table]

    def _create_params(self, query, start, location, job_type ,count=3):
        params = urllib.urlencode({
            # Request parameters
            'publisher': self.key,
            'q': query,
            'limit': count,
            'co': "US",
            'start': start,
            'sort': 'date',
            'st': 'employer',
            'format': 'json',
            'v': 2,
            'l': location,
            'filter':1,
            'jt': job_type
        })
        return params
    
    def search(self, query, start, location, job_type,count=10):
        jobs = []
        prog = tqdm(total=count)
        while not start > count:
            try:
                params = self._create_params(query, start, location,job_type, count)
                response = requests.get(self.endpoint+params)
                response = json.loads(response.text)
                max_result = response['totalResults']
                count = max_result
                print "start= ", start
                print "max-results: " + str(max_result)

                for res in response['results']:
                    if res.get('url'):
                        url = res.get('url')
                        res2 = requests.get(url)
                        jd = ""
                        summary = bs(res2.text,"html.parser")
                        jd =summary.find(class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text.encode("utf-8")
                        '''
                        summary = lxml.html.fromstring(res2.text)
                        for span in summary.xpath('//span[@id="job_summary"]'):
                            for x in span.itertext():
                                #job_sum = x.encode('utf-8')
                                job_sum=x
                                jd += job_sum
                        '''
                        res['job_summary'] = jd
                        
                       # self.db.insert(res)
                    #print " job: ", res['jobtitle'], "\n"
                    jobs.append(res)
                print "start query again"
                start += 25
                clear_output()
                prog.update(25)
                #count -= 25
                
            except Exception as e:
                print e.message
                if start > count: break
                start += 1
                clear_output()
                prog.update(1)
                continue
                
        prog.close()
        return jobs
        
    
    def output_csv(self,filename= None):

        data = self.db.find({})
        data=list(data)
        data=pd.DataFrame(data)
        data=data.drop_duplicates("jobkey")
        
        #data.to_csv(str(filename)+".csv",encoding="utf-8")
        data.to_csv("data.csv",encoding="utf-8")
    def output_excel(self,filename):

        data = self.db.find({})
        data=list(data)
        data=pd.DataFrame(data)
        data=data.drop_duplicates("jobkey")
        data=data.drop("_id",1)
        data.to_excel(str(filename)+".xlsx",encoding="utf-8")

####################################

In [2]:

search_term="entry level quantitative"
location="california"
data_table="indeed_entry_california_research_2018"
job_type="fulltime" #internship, parttime, fulltime
result_count=10000
my_test=IndeedCrawl(data_table)
result = my_test.search(search_term,0,location,job_type,result_count)
result = pd.DataFrame(result)
result.to_csv("data.csv", encoding="utf-8")

  2%|█▊                                                                            | 225/10000 [01:29<54:11,  3.01it/s]


In [6]:
result.iloc[0]["job_summary"]

'$12 an hourTeam Member\nAt Goodwill Industries\xc2\xae Sacramento Valley & Northern Nevada, Inc. (Goodwill) you\xe2\x80\x99ll hear us say \xe2\x80\x9cbecause of Goodwill\xe2\x80\x9d a lot. Often it means, \xe2\x80\x9cbecause of Goodwill I got a hand up\xe2\x80\x9d or \xe2\x80\x9cbecause of Goodwill I have a chance.\xe2\x80\x9d Our organization is built on a rich history of finding the hidden value and possibility in people and donated goods alike since 1933. Today, we continue to innovate the social enterprise of our founders and lead with our mission to utilize Goodwill resources to help people with disadvantages achieve self-sufficiency in everything we do. This mission is expressed through our retail stores and collection sites which create an accessible shopping experience for all, an environmentally conscious alternative to the landfill, and an employment platform for 2,700 people annually. We go beyond creating jobs alone and provide essential resources to our Family of Nonprofi